# IBM Data Science Capstone Project Report

## Identifying The Most Dog-Friendly Neighborhoods in Manhattan, New York City

## 1. Introduction

Whether you are a dog owner or a business in the dog care industry in New York City ("NYC"), identifying locations that are friendly to dogs in the city is always an important mission. According to New York Open Data, an online data platform of the NYC Government, as of 26 July 2019, there are 345,727 dogs with an active license in the city. Among various metropolitans across the world, NYC is perceived as one of the friendliest cities to dogs with a number of parks, dog-run spaces and walkable neighborhoods. For the purpose of this project, the report is going to focus on neighborhoods in Manhattan, the borough with the most dogs, according to New York Open Data.

## 2. Objective

This report aims to evaluate the friendliness of Manhattan's neighborhoods and thus identify the top neighborhoods for dog owners and dog care businesses in this metropolitan. To this end, the report mainly focuses on analyzing the availability of public facilities necessary to dog care across Manhattan's neighborhood, namely parks, dog run and pet facilities.
The report's findings will serve as a helpful source of information for dog owners and businesses in Manhattan in particular and New York City in general, who plan to operate in the dog care industry in this city. 

## 3. Data

To achieve its objective, the report uses the following data:

- The list of neighborhoods in New York City by New York University: https://geo.nyu.edu/catalog/nyu_2451_34572. This list contains 306 neighborhoods across 5 boroughs of New York.
- Longitude and latitude of each of the neighborhood using Python Geocorder package
- Venue data of each of the community areas using Foursquare API. In particular, Foursquare API provides location based experiences with diverse information about venues, users, photos, and check-ins. It supports real time access to placesthat assigns users to specific locations.

## 4. Data Analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium# map rendering library

print('Libraries imported.')



Solving environment: done

# All requested packages already installed.

Libraries imported.


### 4.1 Creating New York Dataframe

#### Load the data

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
nb = newyork_data['features']

#### Transform the data into panda dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)

In [6]:
df.head(10)

,Borough,Neighborhood,Latitude,Longitude


In [7]:
for data in nb:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df = df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
df.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [9]:
# Explore the dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### 4.2 Explore Manhattan Neighborhoods with Foursquare

In [10]:
CLIENT_ID = 'Client ID' # your Foursquare ID
CLIENT_SECRET = 'Client Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 35MHCPGGOHH5SW0NZGTZJ4DOG1G21CMPCRXV1JENWQVUHDMC
CLIENT_SECRET:YT15M1SHE0MKYDRTUTG0LBYAL1NLPW00VUO52LBHMTBSFGRM


In [11]:
# Define the latitude and longitude of New York
latitude = 40.7127281
longitude = -74.0060152

In [12]:
# Define radius and limit
radius = 500
limit = 100

In [13]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    limit)

In [14]:
url

'https://api.foursquare.com/v2/venues/explore?&client_id=35MHCPGGOHH5SW0NZGTZJ4DOG1G21CMPCRXV1JENWQVUHDMC&client_secret=YT15M1SHE0MKYDRTUTG0LBYAL1NLPW00VUO52LBHMTBSFGRM&v=20180605&ll=40.7127281,-74.0060152&radius=500&limit=100'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d73544fcf72a000347cb18e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 173,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57f0689d498e7d49d9189369',
       'name': 'The Bar Room at Temple Court',
       'location': {'address': '123 Nassau St',
        'lat': 40.7114477287544,
        'lng': -74.00680157032005,
        'labe

#### Create a new dataframe for Manhattan

In [16]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [17]:
df_manhattan = df[df['Borough'] == 'Manhattan'].reset_index(drop=True)
df_manhattan.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


#### Get the top venues within the 500 radius

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe for Manhattan venues

In [19]:
manhattan_venues = getNearbyVenues(names=df_manhattan['Neighborhood'],
                                   latitudes=df_manhattan['Latitude'],
                                   longitudes=df_manhattan['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [20]:
print(manhattan_venues.shape)

(3328, 7)


In [21]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


#### Get the list of venues in Manhattan

In [22]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Sh

#### Group rows by neighborhood by taking the mean of the frequency of occurrence of each category

In [23]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Sh

### 4.3 Examine Park Availability in Manhattan

#### 4.3.1 Create a new dataframe for Manhattan parks

In [24]:
manhattan_park = manhattan_grouped[["Neighborhood","Park"]]

In [25]:
manhattan_park.head()

,Neighborhood,Park
0,Battery Park City,0.090000
1,Carnegie Hill,0.000000
2,Central Harlem,0.022727
3,Chelsea,0.010000
4,Chinatown,0.000000


#### 4.3.2 Cluster Neighborhoods

##### Run k-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5
manhattan_clustering = manhattan_park.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 1, 0, 0, 4, 0, 1, 0, 1], dtype=int32)

In [29]:
# Insert cluster labels
manhattan_park.insert(0, 'Cluster Labels', kmeans.labels_)

##### Create a new dataframe with latitude and longitude of each neighborhood

In [31]:
manhattan_park_merged = manhattan_park.copy()

In [32]:
# add latitude/longitude for each neighborhood
manhattan_park_merged = manhattan_park_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

manhattan_park_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,Park,Borough,Latitude,Longitude
0,2,Battery Park City,0.090000,Manhattan,40.711932,-74.016869
1,0,Carnegie Hill,0.000000,Manhattan,40.782683,-73.953256
2,1,Central Harlem,0.022727,Manhattan,40.815976,-73.943211
3,0,Chelsea,0.010000,Manhattan,40.744035,-74.003116
3,0,Chelsea,0.010000,Staten Island,40.594726,-74.189560


##### Visualize the resulting clusters

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_park_merged['Latitude'],manhattan_park_merged['Longitude'], manhattan_park_merged['Neighborhood'], manhattan_park_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 4.3.3 Examine Clusters

##### Cluster 1

In [34]:
manhattan_park_merged.loc[manhattan_park_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighborhood,Park,Borough,Latitude,Longitude
1,0,Carnegie Hill,0.00,Manhattan,40.782683,-73.953256
3,0,Chelsea,0.01,Manhattan,40.744035,-74.003116
3,0,Chelsea,0.01,Staten Island,40.594726,-74.189560
4,0,Chinatown,0.00,Manhattan,40.715618,-73.994279
6,0,Clinton,0.01,Manhattan,40.759101,-73.996119
8,0,East Village,0.01,Manhattan,40.727847,-73.982226
10,0,Flatiron,0.00,Manhattan,40.739673,-73.990947
12,0,Greenwich Village,0.00,Manhattan,40.726933,-73.999914
16,0,Lenox Hill,0.00,Manhattan,40.768113,-73.958860
18,0,Little Italy,0.00,Manhattan,40.719324,-73.997305


##### Cluster 2

In [112]:
manhattan_park_merged.loc[manhattan_park_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighborhood,Park,Borough,Latitude,Longitude
2,1,Central Harlem,0.022727,Manhattan,40.815976,-73.943211
7,1,East Harlem,0.022727,Manhattan,40.792249,-73.944182
9,1,Financial District,0.020000,Manhattan,40.707107,-74.010665
11,1,Gramercy,0.020000,Manhattan,40.737210,-73.981376
14,1,Hudson Yards,0.025641,Manhattan,40.756658,-74.000111
19,1,Lower East Side,0.016667,Manhattan,40.717807,-73.980890
20,1,Manhattan Valley,0.015873,Manhattan,40.797307,-73.964286
23,1,Midtown,0.020000,Manhattan,40.754691,-73.981669
37,1,Washington Heights,0.022989,Manhattan,40.851903,-73.936900
39,1,Yorkville,0.020000,Manhattan,40.775930,-73.947118


##### Cluster 3

In [36]:
manhattan_park_merged.loc[manhattan_park_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighborhood,Park,Borough,Latitude,Longitude
0,2,Battery Park City,0.090,Manhattan,40.711932,-74.016869
25,2,Morningside Heights,0.075,Manhattan,40.808000,-73.963896
30,2,Stuyvesant Town,0.100,Manhattan,40.731000,-73.974052


##### Clutster 4

In [37]:
manhattan_park_merged.loc[manhattan_park_merged['Cluster Labels'] == 3]

,Cluster Labels,Neighborhood,Park,Borough,Latitude,Longitude
21,3,Manhattanville,0.048780,Manhattan,40.816934,-73.957385
32,3,Tribeca,0.050000,Manhattan,40.721522,-74.010683
33,3,Tudor City,0.060976,Manhattan,40.746917,-73.971219


##### Cluter 5

In [38]:
manhattan_park_merged.loc[manhattan_park_merged['Cluster Labels'] == 4]

,Cluster Labels,Neighborhood,Park,Borough,Latitude,Longitude
5,4,Civic Center,0.030000,Manhattan,40.715229,-74.005415
13,4,Hamilton Heights,0.032258,Manhattan,40.823604,-73.949688
15,4,Inwood,0.034483,Manhattan,40.867684,-73.921210
17,4,Lincoln Square,0.030000,Manhattan,40.773529,-73.985338
28,4,Roosevelt Island,0.040000,Manhattan,40.762160,-73.949168
34,4,Turtle Bay,0.030000,Manhattan,40.752042,-73.967708
38,4,West Village,0.040000,Manhattan,40.734434,-74.006180


#### 4.3.4 Examine venues in Stuyvesant Town

It appears Stuyvesant Town is the neighborhood with the highest park concentration in Manhattan. Let's explore this neighborhood further to see in addition to parks, what other venues related to dog care that Stuyvesant Town has to offer. 

##### First, let's take a look at the map of Stuyvesant Town

In [124]:
latitude = 40.731000
longitude = -73.974052

In [126]:
# create map of Stuyvesant Town using latitude and longitude values
map_ri = folium.Map(location=[latitude, longitude], zoom_start=15) 
    
map_ri

The map of Stuyvesant Town shows that this neighborhood indeed has various of parks and outdoor spaces, both large and small. Now let'check the top 30 venues within 1km from Stuyvesant Town and see if it can be a dog-friendly neighborhood.

In [84]:
radius = 1000
limit = 30

In [85]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longtitude, 
    radius, 
    limit)

In [86]:
url

'https://api.foursquare.com/v2/venues/explore?&client_id=35MHCPGGOHH5SW0NZGTZJ4DOG1G21CMPCRXV1JENWQVUHDMC&client_secret=YT15M1SHE0MKYDRTUTG0LBYAL1NLPW00VUO52LBHMTBSFGRM&v=20180605&ll=40.731,-73.971219&radius=1000&limit=30'

In [87]:
# type your answer here

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d735e786001bc0037ef2adf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'New York',
  'headerFullLocation': 'New York',
  'headerLocationGranularity': 'city',
  'totalResults': 147,
  'suggestedBounds': {'ne': {'lat': 40.74000000900001,
    'lng': -73.95936438746722},
   'sw': {'lat': 40.72199999099999, 'lng': -73.98307361253279}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a7b69a3f964a520edea1fe3',
       'name': 'Stuyvesant Cove Park',
       'location': {'address': '20th St and FDR',
        'crossStreet': 'at E 20th St',
        'lat': 40.73251294059505,
        'lng': -73.97387623786926,
    

In [88]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [89]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Stuyvesant Cove Park,Park,40.732513,-73.973876
1,Con Ed Field,Baseball Field,40.729206,-73.973732
2,New York Health & Racquet Club Yacht,Harbor / Marina,40.735590,-73.973973
3,Stuyvesant Town Greenmarket,Farmers Market,40.731429,-73.978222
4,Pouring Ribbons,Cocktail Bar,40.729093,-73.978033
5,The Roost,Bar,40.729084,-73.978638
6,East River Running Path - 23rd St Entrance,Trail,40.735643,-73.974862
7,Oval Kids by apple seeds,Playground,40.731369,-73.977272
8,Transmitter Pier,Scenic Lookout,40.729868,-73.962376
9,Barnyard,Cheese Shop,40.727212,-73.979745


Interestingly, the list above shows that among the 30 top venues within 1km of **Stuyvesant Town**, there are not only parks but also a trail, a scenic lookout, a **pet cafe** and even a **pet service** store! Stuyvesant Town indeed has various options for dogs and their owners to enjoy! 

### 4.4 Examine Dog Run Availability in Manhattan

#### 4.4.1 Create a new dataframe for Manhattan dog run

In [39]:
manhattan_dogrun = manhattan_grouped[["Neighborhood","Dog Run"]]

In [41]:
manhattan_dogrun.head(10)

,Neighborhood,Dog Run
0,Battery Park City,0.00
1,Carnegie Hill,0.00
2,Central Harlem,0.00
3,Chelsea,0.00
4,Chinatown,0.00
5,Civic Center,0.00
6,Clinton,0.01
7,East Harlem,0.00
8,East Village,0.01
9,Financial District,0.00


#### 4.4.2 Cluster neighborhoods

In [42]:
# set number of clusters
kclusters = 5
manhattan_dogrun.insert(0, 'Cluster Labels', kmeans.labels_)
manhattan_dogrun_clustering = manhattan_dogrun.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_dogrun_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 3, 1, 1, 2, 1, 3, 1, 3], dtype=int32)

##### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [45]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
manhattan_dogrun_merged = manhattan_dogrun.copy()


# add clustering labels
manhattan_dogrun_merged["Cluster Labels"] = kmeans.labels_

In [46]:

manhattan_dogrun_merged.head(10)

,Cluster Labels,Neighborhood,Dog Run
0,0,Battery Park City,0.00
1,1,Carnegie Hill,0.00
2,3,Central Harlem,0.00
3,1,Chelsea,0.00
4,1,Chinatown,0.00
5,2,Civic Center,0.00
6,1,Clinton,0.01
7,3,East Harlem,0.00
8,1,East Village,0.01
9,3,Financial District,0.00


In [47]:
# add latitude/longitude for each neighborhood
manhattan_dogrun_merged = manhattan_dogrun_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

manhattan_dogrun_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,Dog Run,Borough,Latitude,Longitude
0,0,Battery Park City,0.0,Manhattan,40.711932,-74.016869
1,1,Carnegie Hill,0.0,Manhattan,40.782683,-73.953256
2,3,Central Harlem,0.0,Manhattan,40.815976,-73.943211
3,1,Chelsea,0.0,Manhattan,40.744035,-74.003116
3,1,Chelsea,0.0,Staten Island,40.594726,-74.189560


##### Visualize the resulting clusters

In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_dogrun_merged['Latitude'],manhattan_dogrun_merged['Longitude'], manhattan_dogrun_merged['Neighborhood'], manhattan_dogrun_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 4.4.3 Examine Clusters

In [49]:
manhattan_dogrun_merged.loc[manhattan_dogrun_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighborhood,Dog Run,Borough,Latitude,Longitude
0,0,Battery Park City,0.0,Manhattan,40.711932,-74.016869
25,0,Morningside Heights,0.0,Manhattan,40.808000,-73.963896
30,0,Stuyvesant Town,0.0,Manhattan,40.731000,-73.974052


In [50]:
manhattan_dogrun_merged.loc[manhattan_dogrun_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighborhood,Dog Run,Borough,Latitude,Longitude
1,1,Carnegie Hill,0.00,Manhattan,40.782683,-73.953256
3,1,Chelsea,0.00,Manhattan,40.744035,-74.003116
3,1,Chelsea,0.00,Staten Island,40.594726,-74.189560
4,1,Chinatown,0.00,Manhattan,40.715618,-73.994279
6,1,Clinton,0.01,Manhattan,40.759101,-73.996119
8,1,East Village,0.01,Manhattan,40.727847,-73.982226
10,1,Flatiron,0.00,Manhattan,40.739673,-73.990947
12,1,Greenwich Village,0.00,Manhattan,40.726933,-73.999914
16,1,Lenox Hill,0.00,Manhattan,40.768113,-73.958860
18,1,Little Italy,0.00,Manhattan,40.719324,-73.997305


In [51]:
manhattan_dogrun_merged.loc[manhattan_dogrun_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighborhood,Dog Run,Borough,Latitude,Longitude
5,2,Civic Center,0.000000,Manhattan,40.715229,-74.005415
13,2,Hamilton Heights,0.000000,Manhattan,40.823604,-73.949688
15,2,Inwood,0.017241,Manhattan,40.867684,-73.921210
17,2,Lincoln Square,0.000000,Manhattan,40.773529,-73.985338
28,2,Roosevelt Island,0.040000,Manhattan,40.762160,-73.949168
34,2,Turtle Bay,0.010000,Manhattan,40.752042,-73.967708
38,2,West Village,0.000000,Manhattan,40.734434,-74.006180


In [52]:
manhattan_dogrun_merged.loc[manhattan_dogrun_merged['Cluster Labels'] == 3]

,Cluster Labels,Neighborhood,Dog Run,Borough,Latitude,Longitude
2,3,Central Harlem,0.000000,Manhattan,40.815976,-73.943211
7,3,East Harlem,0.000000,Manhattan,40.792249,-73.944182
9,3,Financial District,0.000000,Manhattan,40.707107,-74.010665
11,3,Gramercy,0.000000,Manhattan,40.737210,-73.981376
14,3,Hudson Yards,0.025641,Manhattan,40.756658,-74.000111
19,3,Lower East Side,0.000000,Manhattan,40.717807,-73.980890
20,3,Manhattan Valley,0.015873,Manhattan,40.797307,-73.964286
23,3,Midtown,0.000000,Manhattan,40.754691,-73.981669
37,3,Washington Heights,0.000000,Manhattan,40.851903,-73.936900
39,3,Yorkville,0.010000,Manhattan,40.775930,-73.947118


In [53]:
manhattan_dogrun_merged.loc[manhattan_dogrun_merged['Cluster Labels'] == 4]

,Cluster Labels,Neighborhood,Dog Run,Borough,Latitude,Longitude
21,4,Manhattanville,0.00000,Manhattan,40.816934,-73.957385
32,4,Tribeca,0.01000,Manhattan,40.721522,-74.010683
33,4,Tudor City,0.02439,Manhattan,40.746917,-73.971219


#### 4.4.4 Examine Venues in Roosevelt Island

Similar to what we did with Stuyvesant Town, let's first take a look at the map of Roosevelt Island to check if it actually has a signigicant amount of dog runs

In [90]:
ri_latitude = 40.762160
ri_longitude = -73.949168

In [133]:
# create map of Rosevelt Island using latitude and longitude values
map_ri = folium.Map(location=[ri_latitude, ri_longitude], zoom_start=15) 
    
map_ri

Yes, the amount of open spaces as we see on the map of the island seems to support our finding with Roosevelt Island having the most dog runs. 
Now, let's examine other venues nearby the island as well!

In [114]:
radius = 1000
limit = 30

In [115]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ri_latitude, 
    ri_longitude, 
    radius, 
    limit)

In [116]:
url

'https://api.foursquare.com/v2/venues/explore?&client_id=35MHCPGGOHH5SW0NZGTZJ4DOG1G21CMPCRXV1JENWQVUHDMC&client_secret=YT15M1SHE0MKYDRTUTG0LBYAL1NLPW00VUO52LBHMTBSFGRM&v=20180605&ll=40.76216,-73.949168&radius=1000&limit=30'

In [117]:
# type your answer here

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d736cb6d69ed0002cbf5e01'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Roosevelt Island',
  'headerFullLocation': 'Roosevelt Island, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 136,
  'suggestedBounds': {'ne': {'lat': 40.77116000900001,
    'lng': -73.9373078316877},
   'sw': {'lat': 40.75315999099999, 'lng': -73.9610281683123}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4db73c8a5da3b5fa68a6628b',
       'name': 'Roosevelt Island Promenade',
       'location': {'address': 'Roosevelt island',
        'lat': 40.76087483014197,
        'lng': -73.95243668606233,
    

In [118]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [119]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Roosevelt Island Promenade,Outdoors & Recreation,40.760875,-73.952437
1,NISI,Greek Restaurant,40.761915,-73.949966
2,Manhattan Park,Park,40.765540,-73.947075
3,East Side River Walk,Waterfront,40.765391,-73.951725
4,Sotheby's,Art Gallery,40.766494,-73.954003
5,Tanoshi Sushi,Sushi Restaurant,40.767747,-73.953203
6,Roosevelt Island Tram (Roosevelt Island Station),Tram Station,40.757649,-73.954461
7,Roosevelt Island Farmer's Market,Farmers Market,40.764210,-73.947606
8,East River Esplanade - E 63rd St,Park,40.760773,-73.956839
9,East River Walk,Park,40.760748,-73.956099


The above list shows that ***7 out of top 30 venues nearby Roosevelt Island are open spaces*** , including 4 parks, a scenic lookout, a recreation park and a water front! What a good candidate for a dog-friendly neighborhood in a concrete jungle like New York city! 